In [1]:
import random

import numpy as np
from sklearn.cross_validation import train_test_split  
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import load_model
from keras import backend as K
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50

from load_face_dataset import load_dataset



C:\Users\Jacky\AppData\Local\Continuum\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Jacky\AppData\Local\Continuum\anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class Dataset:
    def __init__(self,path_name,path_name2,):
        #训练集
        self.train_images = None
        self.train_labels = None
        
        #验证集
        self.valid_images = None
        self.valid_labels = None
        
        #测试集
        self.test_images  = None            
        self.test_labels  = None
        
        #数据集加载路径
        self.path_name = path_name
        self.path_name2 = path_name2

        #当前库采用的维度顺序
        self.input_shape = None
        
    #加载数据集并按照交叉验证的原则划分数据集并进行相关预处理工作
    def load(self,n,img_rows = 224, img_cols = 224, 
             img_channels = 3, nb_classes = 5):
        #########

        a,b,c,d,e,f = load_dataset(self.path_name,self.path_name2,n)
        
        train_images = a.astype('float32')
        valid_images = c.astype('float32')
        test_images = e.astype('float32')
        
        train_images = preprocess_input_resnet50(train_images)
        valid_images = preprocess_input_resnet50(valid_images)
        test_images = preprocess_input_resnet50(test_images)
        
        train_labels = b
        valid_labels = d
        test_labels = f
        
        
        train_labels = np_utils.to_categorical(train_labels, nb_classes)                        
        valid_labels = np_utils.to_categorical(valid_labels, nb_classes)            
        test_labels = np_utils.to_categorical(test_labels, nb_classes)
        print("train image shape: ")
        print(train_images.shape)
        self.train_images = train_images
        self.valid_images = valid_images
        self.test_images  = test_images
        self.train_labels = train_labels
        self.valid_labels = valid_labels
        self.test_labels  = test_labels
        


In [3]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
                featurewise_center = False,             #是否使输入数据去中心化（均值为0），
                samplewise_center  = False,             #是否使输入数据的每个样本均值为0
                featurewise_std_normalization = False,  #是否数据标准化（输入数据除以数据集的标准差）
                samplewise_std_normalization  = False,  #是否将每个样本数据除以自身的标准差
                zca_whitening = False,                  #是否对输入数据施以ZCA白化
                rotation_range = 20,                    #数据提升时图片随机转动的角度(范围为0～180)
                width_shift_range  = 0.2,               #数据提升时图片水平偏移的幅度（单位为图片宽度的占比，0~1之间的浮点数）
                height_shift_range = 0.2,               #同上，只不过这里是垂直
                horizontal_flip = True,                 #是否进行随机水平翻转
                vertical_flip = False)                  #是否进行随机垂直翻转


In [4]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import regularizers
from keras.callbacks import ModelCheckpoint  
from keras.optimizers import SGD
from keras.optimizers import Adam, Adamax
from sklearn.metrics import accuracy_score

ac = []

n=-1
while(n<10):
    n=n+1

    a=0
#dataset = Dataset('C:/Users/Jacky/Desktop/stanford_dog/crop_resize/','C:/Users/Jacky/Desktop/stanford_dog/data/2/')
    #pt = 'C:/Users/Jacky/Desktop/dogtest_new/data/'+str(n)+'/'
    pt = 'C:/Users/Jacky/Desktop/dogtest_new/data/t'+str(n)+'/'
    #spt = 'E:/saved_models/mydata_head'+str(m)+'_'+str(n)+'.hdf5'
    spt = 'E:/saved_models/mydatatype_head_'+str(n)+'.hdf5'
    dpt = 'C:/Users/Jacky/Desktop/id_dog/'
    #dpt = 'C:/Users/Jacky/Desktop/id_dog/'+str(m)+'/'
    print(pt)
    print(dpt)
    
    dataset = Dataset(dpt,pt)
    dataset.load(a)
    
    datagen.fit(dataset.train_images)


    Resmodel = ResNet50(weights='imagenet', include_top = False ,input_shape = (224,224,3))
#Resmodel = ResNet50(weights=None, include_top = False ,input_shape = (224,224,3))

#Resmodel.load_weights('D:/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    x=0
    for layer in Resmodel.layers:
        x+=1
        layer.trainable = True
    

    y = Resmodel.output
    y = GlobalAveragePooling2D()(y)

    y = Dense(1024,kernel_regularizer=regularizers.l2(0.001),activation='relu')(y)
    y = Dropout(0.5)(y)
    y = Dense(5, activation='softmax')(y)
    model = Model(inputs=Resmodel.input, outputs=y)




    EPOCHS = 20
    sgd = SGD(lr = 0.001, decay = 1e-6,momentum = 0.9, nesterov = True)

#model.load_weights('E:/saved_models/stanford_all_train.hdf5')

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    checkpointer = ModelCheckpoint(filepath=spt,monitor='val_loss', 
                                   verbose=1, save_best_only=True, mode='min')

    model.fit_generator(datagen.flow(dataset.train_images, dataset.train_labels, batch_size = 16),
                                     validation_data = (dataset.valid_images, dataset.valid_labels),
                                     samples_per_epoch = dataset.train_images.shape[0],
                                     epochs = EPOCHS,callbacks=[checkpointer],verbose=2)
    
   # model.fit(dataset.train_images, dataset.train_labels,
   #           validation_data=(dataset.valid_images, dataset.valid_labels),
   #           epochs=EPOCHS,batch_size=64,callbacks=[checkpointer],verbose=2)

    model.load_weights(spt)

    predictions = model.predict(dataset.test_images)
    breed_predictions = [np.argmax(prediction) for prediction in predictions]
    breed_true_labels = [np.argmax(true_label) for true_label in dataset.test_labels]
    aa = accuracy_score(breed_true_labels, breed_predictions) * 100
    ac.append(aa)
    print('Test accuracy: %.4f%%' % (aa))


C:/Users/Jacky/Desktop/dogtest_new/data/t0/
C:/Users/Jacky/Desktop/id_dog/


100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:15<00:00, 39.69it/s]


image shape: 
(2500, 224, 224, 3)
train image shape: 
(2500, 224, 224, 3)


C:\Users\Jacky\AppData\Local\Continuum\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
C:\Users\Jacky\AppData\Local\Continuum\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:69: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=156, validation_data=(array([[[..., callbacks=[<keras.ca..., epochs=20, verbose=2)`


Epoch 1/20
 - 32s - loss: 1.7285 - acc: 0.8682 - val_loss: 1.4347 - val_acc: 0.9744

Epoch 00001: val_loss improved from inf to 1.43466, saving model to E:/saved_models/mydatatype_head_0.hdf5
Epoch 2/20
 - 24s - loss: 1.4412 - acc: 0.9720 - val_loss: 1.3818 - val_acc: 0.9872

Epoch 00002: val_loss improved from 1.43466 to 1.38183, saving model to E:/saved_models/mydatatype_head_0.hdf5
Epoch 3/20
 - 24s - loss: 1.4188 - acc: 0.9792 - val_loss: 1.4113 - val_acc: 0.9696

Epoch 00003: val_loss did not improve from 1.38183
Epoch 4/20
 - 24s - loss: 1.3869 - acc: 0.9848 - val_loss: 1.3536 - val_acc: 0.9920

Epoch 00004: val_loss improved from 1.38183 to 1.35355, saving model to E:/saved_models/mydatatype_head_0.hdf5
Epoch 5/20
 - 24s - loss: 1.3712 - acc: 0.9864 - val_loss: 1.3429 - val_acc: 0.9968

Epoch 00005: val_loss improved from 1.35355 to 1.34292, saving model to E:/saved_models/mydatatype_head_0.hdf5
Epoch 6/20
 - 24s - loss: 1.3570 - acc: 0.9892 - val_loss: 1.3366 - val_acc: 0.9952


...

100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:17<00:00, 35.53it/s]


image shape: 
(2500, 224, 224, 3)
train image shape: 
(2500, 224, 224, 3)
Epoch 1/20
 - 45s - loss: 1.7473 - acc: 0.8610 - val_loss: 1.4295 - val_acc: 0.9744

Epoch 00001: val_loss improved from inf to 1.42955, saving model to E:/saved_models/mydatatype_head_8.hdf5
Epoch 2/20
 - 29s - loss: 1.4483 - acc: 0.9647 - val_loss: 1.4488 - val_acc: 0.9728

Epoch 00002: val_loss did not improve from 1.42955
Epoch 3/20
 - 29s - loss: 1.3978 - acc: 0.9808 - val_loss: 1.4634 - val_acc: 0.9648

Epoch 00003: val_loss did not improve from 1.42955
Epoch 4/20
 - 29s - loss: 1.3753 - acc: 0.9876 - val_loss: 1.3754 - val_acc: 0.9888

Epoch 00004: val_loss improved from 1.42955 to 1.37545, saving model to E:/saved_models/mydatatype_head_8.hdf5
Epoch 5/20
 - 29s - loss: 1.3634 - acc: 0.9900 - val_loss: 1.3647 - val_acc: 0.9920

Epoch 00005: val_loss improved from 1.37545 to 1.36475, saving model to E:/saved_models/mydatatype_head_8.hdf5
Epoch 6/20
 - 29s - loss: 1.3595 - acc: 0.9884 - val_loss: 1.3649 - va

100%|████████████████████████████████████████████████████████████████████████████████| 625/625 [00:17<00:00, 35.87it/s]


image shape: 
(2500, 224, 224, 3)
train image shape: 
(2500, 224, 224, 3)
Epoch 1/20
 - 48s - loss: 1.7503 - acc: 0.8614 - val_loss: 1.3998 - val_acc: 0.9888

Epoch 00001: val_loss improved from inf to 1.39984, saving model to E:/saved_models/mydatatype_head_9.hdf5
Epoch 2/20
 - 30s - loss: 1.4417 - acc: 0.9772 - val_loss: 1.3694 - val_acc: 0.9968

Epoch 00002: val_loss improved from 1.39984 to 1.36942, saving model to E:/saved_models/mydatatype_head_9.hdf5
Epoch 3/20
 - 30s - loss: 1.4073 - acc: 0.9816 - val_loss: 1.3571 - val_acc: 0.9936

Epoch 00003: val_loss improved from 1.36942 to 1.35714, saving model to E:/saved_models/mydatatype_head_9.hdf5
Epoch 4/20
 - 30s - loss: 1.3831 - acc: 0.9856 - val_loss: 1.3840 - val_acc: 0.9888

Epoch 00004: val_loss did not improve from 1.35714
Epoch 5/20
 - 29s - loss: 1.3668 - acc: 0.9892 - val_loss: 1.3536 - val_acc: 0.9952

Epoch 00005: val_loss improved from 1.35714 to 1.35357, saving model to E:/saved_models/mydatatype_head_9.hdf5
Epoch 6/20

FileNotFoundError: File b'C:/Users/Jacky/Desktop/dogtest_new/data/t10/train.csv' does not exist

In [5]:
print(ac)

[98.88, 99.68, 99.2, 99.2, 99.03999999999999, 98.72, 99.36, 99.2, 99.36, 99.2]


In [ ]:
    model.load_weights(spt)

    predictions = model.predict(dataset.test_images)
    breed_predictions = [np.argmax(prediction) for prediction in predictions]
    breed_true_labels = [np.argmax(true_label) for true_label in dataset.test_labels]
    aa = accuracy_score(breed_true_labels, breed_predictions) * 100
    ac.append(aa)
    print('Test accuracy: %.4f%%' % (aa))